# 0. Steps<a name="major steps"></a>
<b> Detection of Acadja installations in Nokoue lake, Benin using Sentinel 1 data </b>
* Data used: The Sentinel-1 mission provides data from a dual-polarization C-band Synthetic Aperture Radar (SAR) instrument at 5.405GHz (C band). In this study, we used the S1 Ground Range Detected (GRD) scenes, processed using the Sentinel-1 Toolbox to generate a calibrated, ortho-corrected product. The scenes available as image collection in GEE is used.  
* Selection of polarisation used: The backscatter coefficient in dB in the Vertical Transmit and Horizontal receive (VH) mode is used. The VV mode has a striping due to tiling effect within the lake, hence could not use. VV modes tend to pick up height/vertical features where VH modes tend to be more sensitive to surface textures. Hence for the Acadja detection VH mode could be more suitable based on the variation in texture over water body due to the wood installations.
* Method: Thresholding based on VH mode data was implemented. The analysis of S1 VH data over the year 2022 showed a clear pattern of acadja's within the lake. Median of all the VH mode data over 2022 was computed and applied a minimum and maximum thresholds of -24.5 and -22 respectively. The final binary map is displayed in the interactive map below.

# 1. Installation of required library<a name="install"></a>
You need to have geemap python library installed in your computer. <br>
If you are using conda environment then use the below command to install geemap. <br>
<b>This is one time step.</b>

In [1]:
#conda install -c conda-forge geemap

<b>OR</b> use pip to install the geemap package as given below

In [2]:
#!pip install --upgrade geemap

In [3]:
#!conda install -c conda-forge geopandas

In [4]:
#geemap.update_package()

In [5]:
#!conda activate base

# 2. Initializing the GEE<a name="initial"></a>
Let us start with geemap by importing the library and bringing in a interactive map.

In [19]:
import os
import ee
import geemap
#import geopandas

Now you will define a interactive map and get it inline in this notebook. <br>
<b> Note that first time you do this step in your computer, it will ask for authorize the access needed by GEE. You have to follow the instructions and finish the authorization. This is only one time in local computer</b> <br>
See this [youtube video](https://www.youtube.com/watch?v=h0pz3S6Tvx0&t=359s) showing the authorization steps.

Now that you have successfully initiated the geemap, let us explore data sets from GEE by importing them to this notebook.

# 3. Import data in GEE<a name="import"></a>
GEE offers access to petabytes of data from various data sources primarily from satellites and remote sensing.<br>
You can check [this link](https://developers.google.com/earth-engine/datasets) for the data catalog of GEE.<br>
For each type of data, the catalog gives you <b>"Earthengine snippet"</b> as shown below (only an example) which can be used to access this particular data. <br>

In [88]:
s1 = ee.ImageCollection("COPERNICUS/S1_GRD");
lakes = ee.FeatureCollection("users/spareeth/Benin/Nokoue_NW");
fld_all = ee.FeatureCollection("users/spareeth/Benin/fieldata");
fld_ac = ee.FeatureCollection("users/spareeth/Benin/fieldata_acadja");
lake_bnd = ee.FeatureCollection("users/spareeth/Benin/lake_bnd");

In [97]:
lake = lakes.filter("name == 'nokoue'")

In [132]:
vh_2022_raw = ee.ImageCollection("COPERNICUS/S1_GRD") \
            .filterBounds(lake_bnd) \
            .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH')) \
            .filter(ee.Filter.eq('instrumentMode', 'IW')) \
            .filterDate('2022-01-01', '2022-12-31') \
            .median() \
            .clip(lake);

Compute Radar Vegetation Index (RVI) from sentinel 1 GRD data. Read about RVI [here](https://custom-scripts.sentinel-hub.com/custom-scripts/sentinel-1/radar_vegetation_index/).
Note that this is not used for Acadja extraction due to striped effect.

In [133]:
rvi = vh_2022.expression(
      '4 * VH / (VH + VV)', {
      'VH': vh_2022.select('VH'),
      'VV': vh_2022.select('VV')
});

In [134]:
VH = vh_2022.select('VH').clip(lake);

In [135]:
acadja = VH.lt(-22).And(VH.gt(-24.5));

In [136]:
Map = geemap.Map(basemap="HYBRID")
Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

In [137]:
Map.center_object(lake)

In [138]:
Viz = {'min': -26, 'max': -18, 'palette': ['green', 'yellow', 'brown']}

In [139]:
Map.addLayer(vh_2022.select('VH'), Viz, 'VH', 1);

In [140]:
Map.addLayer(acadja.selfMask(), {}, 'Acadja');

In [141]:
Map.addLayer(fld_ac, {}, 'field acadja', 1);